## Use this notebook to take water bodies from the HydroPolys layer and add them to the Named_Reservoirs shapefile

In [1]:
%matplotlib inline

import sys

sys.path.append('Scripts')
from datahandling import wofs_fuser

import datacube
import numpy as np
import pandas as pd
import xarray as xr
import matplotlib.pyplot as plt
from pathlib import Path
from datacube.utils import masking
from datacube.utils import geometry 
from datacube.utils.geometry import CRS
from datacube.helpers import write_geotiff
import geopandas as gpd
import ipyleaflet as ipyl
import ipywidgets as ipyw
import json



dc = datacube.Datacube()

/env/lib/python3.6/site-packages/geopandas/_compat.py:88: UserWarning: The Shapely GEOS version (3.7.2-CAPI-1.11.0 ) is incompatible with the GEOS version PyGEOS was compiled with (3.9.0-CAPI-1.16.2). Conversions between both will be slow.
  shapely_geos_version, geos_capi_version_string


## Read and fomat the polygon you want to add to the Named_Reservoirs shapefile

In [2]:
csv = 'Library/MURRAY_2_PONDAGE_401571.csv'
gdf = gpd.read_file('HydroPolys/MURRAY_2_PONDAGE.shp')
gdf

,NAME,SHAPE_Leng,SHAPE_Area,Area_calc,lat,lon,geometry
0,MURRAY 2 PONDAGE,0.054875,0.000014,136624.041,-36.241612,148.185751,"POLYGON ((148.18139 -36.24036, 148.18151 -36.2..."


In [3]:
#Get the centroid location of the reservoir

gdf = gdf.to_crs(epsg=4326) #4326 is the tag for lat lon format
#we need to get the lat and lon of where the dam is to call the right satellite data later
#we want the lat and lon to be columns in the data to make it easy when we query the wofs data later
#Getting the centroid from the polygon and adding a latitude and longitude column (I found this code on the internet):
def getXY(pt):
    return (pt.x, pt.y)
centroidseries = gdf['geometry'].centroid
x,y = [list(t) for t in zip(*map(getXY, centroidseries))]

gdf = gdf.assign(lon=x)
gdf = gdf.assign(lat=y)
gdf = gdf[['NAME', 'SHAPE_Leng', 'SHAPE_Area', 'lat', 'lon', 'geometry']]
gdf

/env/lib/python3.6/site-packages/ipykernel_launcher.py:9: UserWarning: Geometry is in a geographic CRS. Results from 'centroid' are likely incorrect. Use 'GeoSeries.to_crs()' to re-project geometries to a projected CRS before this operation.

  if __name__ == '__main__':


,NAME,SHAPE_Leng,SHAPE_Area,lat,lon,geometry
0,MURRAY 2 PONDAGE,0.054875,0.000014,-36.241612,148.185751,"POLYGON ((148.18139 -36.24036, 148.18151 -36.2..."


In [6]:
gauge_number_df

,Station Long Name,gauge_ID
0,Station Number,401571.000000
1,Station Location Latitude,-36.237728
2,Station Location Longitude,148.181153


In [9]:
gauge_number_df = pd.read_csv(csv, nrows=3, escapechar='#')
column2 = list(gauge_number_df)[1]
gauge_number_df = gauge_number_df.rename(columns = {column2 : 'gauge_ID'})
ID_str = gauge_number_df.at[0, 'gauge_ID']
ID_str = ID_str.astype(int)
ID_str = ID_str.astype(str)
lat = gauge_number_df.at[1, 'gauge_ID']
lon = gauge_number_df.at[2, 'gauge_ID']
gdf['gauge_ID'] = ID_str
gdf['lat_gauge'] = lat
gdf['lon_gauge'] = lon
gdf['Area_calc'] = ''
gdf['staion_nam'] = column2
gdf = gdf[['NAME', 'SHAPE_Leng', 'SHAPE_Area', 'lat', 'lon', 'Area_calc', 'staion_nam', 'gauge_ID', 'lat_gauge', 'lon_gauge', 'geometry']]
gdf

,NAME,SHAPE_Leng,SHAPE_Area,lat,lon,Area_calc,staion_nam,gauge_ID,lat_gauge,lon_gauge,geometry
0,MURRAY 2 PONDAGE,0.054875,0.000014,-36.241612,148.185751,,Khancoban Back Creek at Murray 2 Pondage,401571,-36.237728,148.181153,"POLYGON ((148.18139 -36.24036, 148.18151 -36.2..."


In [10]:
gdf2 = gpd.read_file('Named_Reservoirs/Named_Reservoirs7.shp')
gdf2 = gdf2.to_crs(epsg=4326)
gdf2

,NAME,SHAPE_Leng,SHAPE_Area,lat,lon,Area_calc,staion_nam,gauge_ID,lat_gauge,lon_gauge,geometry
0,WILSON RESERVOIR,0.034915,0.000028,-37.510018,144.018549,270561.736,LAL LAL C WILSON RES,232220A,-37.5144,144.0177,"POLYGON ((144.01821 -37.51455, 144.01719 -37.5..."
1,SUGARLOAF RESERVOIR,0.212903,0.000438,-37.672015,145.305467,4284887.27,SUGARLOAF RES DAM,229607A,-37.6724,145.279,"POLYGON ((145.31084 -37.68275, 145.31059 -37.6..."
2,SUGARLOAF RESERVOIR,0.212903,0.000438,-37.672015,145.305467,4284887.27,SUGARLOAF RES. H.G.,408217A,-37.103981,143.402094,"POLYGON ((145.31084 -37.68275, 145.31059 -37.6..."
3,GREEN LAKE,0.155045,0.000715,-36.435338,144.834187,7111145.66,Greens Lake,sp-o10454,-36.433798,144.834474,"POLYGON ((144.82094 -36.43662, 144.82081 -36.4..."
4,GREEN LAKE,0.075718,0.000170,-36.792870,142.292810,1688188.34,Green Lake,GREEN,-36.79225,142.2921,"POLYGON ((142.29212 -36.80000, 142.29128 -36.7..."
...,...,...,...,...,...,...,...,...,...,...,...
421,KIRAR WEIR,0.257613,0.000240,-25.299279,151.063109,2681688.02,Kirar Weir HW,136121A,-25.34556,151.079067,"POLYGON ((151.07836 -25.34552, 151.07834 -25.3..."
422,MARIAN WEIR,0.266732,0.000134,-21.146641,148.885758,1544632.18,Marian Weir HW,125003A,-21.141222,148.936264,"POLYGON ((148.82987 -21.17723, 148.82926 -21.1..."
423,MARIAN WEIR,0.266732,0.000134,-21.146641,148.885758,1544632.18,Mirani Weir HW,125008A,-21.17788,148.83,"POLYGON ((148.82987 -21.17723, 148.82926 -21.1..."
424,TINANA BARRAGE,0.034002,0.000012,-25.571684,152.713346,129235.966,Tinana Barrage HW,138008A,-25.57206,152.7181,"POLYGON ((152.71974 -25.56929, 152.71973 -25.5..."


In [11]:
gdf3 = gdf2.append(gdf)
gdf3

,NAME,SHAPE_Leng,SHAPE_Area,lat,lon,Area_calc,staion_nam,gauge_ID,lat_gauge,lon_gauge,geometry
0,WILSON RESERVOIR,0.034915,0.000028,-37.510018,144.018549,270561.736,LAL LAL C WILSON RES,232220A,-37.5144,144.0177,"POLYGON ((144.01821 -37.51455, 144.01719 -37.5..."
1,SUGARLOAF RESERVOIR,0.212903,0.000438,-37.672015,145.305467,4284887.27,SUGARLOAF RES DAM,229607A,-37.6724,145.279,"POLYGON ((145.31084 -37.68275, 145.31059 -37.6..."
2,SUGARLOAF RESERVOIR,0.212903,0.000438,-37.672015,145.305467,4284887.27,SUGARLOAF RES. H.G.,408217A,-37.103981,143.402094,"POLYGON ((145.31084 -37.68275, 145.31059 -37.6..."
3,GREEN LAKE,0.155045,0.000715,-36.435338,144.834187,7111145.66,Greens Lake,sp-o10454,-36.433798,144.834474,"POLYGON ((144.82094 -36.43662, 144.82081 -36.4..."
4,GREEN LAKE,0.075718,0.000170,-36.792870,142.292810,1688188.34,Green Lake,GREEN,-36.79225,142.2921,"POLYGON ((142.29212 -36.80000, 142.29128 -36.7..."
...,...,...,...,...,...,...,...,...,...,...,...
422,MARIAN WEIR,0.266732,0.000134,-21.146641,148.885758,1544632.18,Marian Weir HW,125003A,-21.141222,148.936264,"POLYGON ((148.82987 -21.17723, 148.82926 -21.1..."
423,MARIAN WEIR,0.266732,0.000134,-21.146641,148.885758,1544632.18,Mirani Weir HW,125008A,-21.17788,148.83,"POLYGON ((148.82987 -21.17723, 148.82926 -21.1..."
424,TINANA BARRAGE,0.034002,0.000012,-25.571684,152.713346,129235.966,Tinana Barrage HW,138008A,-25.57206,152.7181,"POLYGON ((152.71974 -25.56929, 152.71973 -25.5..."
425,LAKE GLENMAGGIE,0.550671,0.001746,-37.906605,146.768497,None,MACALISTER RIVER @ LAKE GLENMAGGIE (HEAD GAUGE),225225A,-37.90833333,146.8,"POLYGON ((146.72768 -37.85241, 146.72792 -37.8..."


In [12]:
gdf3.to_file('Named_Reservoirs8.shp')